<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# Practice Project

Estimated time needed: **60** minutes

This practice project focuses on data transformation and integration using PySpark. You will work with two datasets, perform various transformations such as adding columns, renaming columns, dropping unnecessary columns, joining dataframes, and finally, writing the results into both a Hive warehouse and an HDFS file system.


### Prerequisites 

For this lab assignment, you will use wget, Python and Spark (PySpark). Therefore, it's essential to make sure that the below-specified libraries are installed in your lab environment or within Skills Network (SN) Labs.  

 


In [2]:
# Installing required packages

!pip install wget pyspark  findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 MB 1.5 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 34.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.4.2-py2.py3-none-any.whl size=311619841 sha256=266d89086db14a4d669c73275553b95a8c68d10ef87cadb95a2d865e54328218
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/c3/8a/ac/cd39777597318310141c8a783c06f516815a66194f100f96b6
Successfully built pyspark


#### Prework - Initiate the Spark Session


In [3]:
import findspark

findspark.init()

In [4]:
# PySpark is the Spark API for Python. In this lab, we use PySpark to initialize the SparkContext.   

from pyspark import SparkContext, SparkConf

from pyspark.sql import SparkSession

In [5]:
# Creating a SparkContext object

sc = SparkContext.getOrCreate()

# Creating a Spark Session

spark = SparkSession \
    .builder \
    .appName("Python Spark DataFrames basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

24/01/30 06:33:59 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Task 1: Load datasets into PySpark DataFrames

Download the datasets from the folloing links using `wget` and load it in a Spark Dataframe.

1. https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/data/dataset1.csv  
2. https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/data/dataset2.csv  

*Hint: Import wget*


In [6]:
import os
os.getcwd()

'/resources/labs/authoride/IBMSkillsNetwork+BD0225EN/labs'

In [7]:
os.listdir()

['._SUCCESS.crc',
 '.ipynb_checkpoints',
 '.part-00000-9da5cbc2-6777-47e8-b199-9c4f5e28891e-c000.snappy.parquet.crc',
 '.part-00004-9da5cbc2-6777-47e8-b199-9c4f5e28891e-c000.snappy.parquet.crc',
 '.part-00009-9da5cbc2-6777-47e8-b199-9c4f5e28891e-c000.snappy.parquet.crc',
 '.part-00012-9da5cbc2-6777-47e8-b199-9c4f5e28891e-c000.snappy.parquet.crc',
 '.part-00018-9da5cbc2-6777-47e8-b199-9c4f5e28891e-c000.snappy.parquet.crc',
 '.part-00022-9da5cbc2-6777-47e8-b199-9c4f5e28891e-c000.snappy.parquet.crc',
 '.part-00025-9da5cbc2-6777-47e8-b199-9c4f5e28891e-c000.snappy.parquet.crc',
 '.part-00026-9da5cbc2-6777-47e8-b199-9c4f5e28891e-c000.snappy.parquet.crc',
 '.part-00028-9da5cbc2-6777-47e8-b199-9c4f5e28891e-c000.snappy.parquet.crc',
 '.part-00031-9da5cbc2-6777-47e8-b199-9c4f5e28891e-c000.snappy.parquet.crc',
 '.part-00036-9da5cbc2-6777-47e8-b199-9c4f5e28891e-c000.snappy.parquet.crc',
 '.part-00045-9da5cbc2-6777-47e8-b199-9c4f5e28891e-c000.snappy.parquet.crc',
 '.part-00046-9da5cbc2-6777-47e8-b1

In [5]:
#download dataset using wget
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/data/dataset1.csv >> dataset1.csv

--2024-01-28 07:09:30--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/data/dataset1.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104, 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4115 (4.0K) [text/csv]
Saving to: ‘dataset1.csv.1’

dataset1.csv.1      100%[===================>]   4.02K  --.-KB/s    in 0s      

2024-01-28 07:09:30 (35.6 MB/s) - ‘dataset1.csv.1’ saved [4115/4115]




<details>
    <summary>Click here for Solution</summary>

```python
# download the dataset using wget
import wget

link_to_data1 = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/data/dataset1.csv'
wget.download(link_to_data1)

link_to_data2 = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/data/dataset2.csv'
wget.download(link_to_data2)
```

</details>


In [23]:
!curl https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/data/dataset1.csv >> dataset1.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4115  100  4115    0     0  60514      0 --:--:-- --:--:-- --:--:-- 60514


In [20]:
!curl https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/data/dataset2.csv >> dataset2.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2688  100  2688    0     0  25846      0 --:--:-- --:--:-- --:--:-- 28595


In [109]:
#load the data into a pyspark dataframe

df1 = spark.read.csv('dataset1.csv',header = True)
df2 = spark.read.csv('dataset2.csv',header = True)


<details>
    <summary>Click here for Solution</summary>

```python

#load the data into a pyspark dataframe
    
df1 = spark.read.csv("dataset1.csv", header=True, inferSchema=True)
df2 = spark.read.csv("dataset2.csv", header=True, inferSchema=True)
```

</details>




### Task 2: Display the schema of both dataframes

Display the schema of `df1` and `df2` to understand the structure of the datasets.


In [9]:
df1.show()

+-----------+-----------+------+-----------+--------+
|customer_id|date_column|amount|description|location|
+-----------+-----------+------+-----------+--------+
|          1|   1/1/2022|  5000| Purchase A| Store A|
|          2|  15/2/2022|  1200| Purchase B| Store B|
|          3|  20/3/2022|   800| Purchase C| Store C|
|          4|  10/4/2022|  3000| Purchase D| Store D|
|          5|   5/5/2022|  6000| Purchase E| Store E|
|          6|  10/6/2022|  4500| Purchase F| Store F|
|          7|  15/7/2022|   200| Purchase G| Store G|
|          8|  20/8/2022|  3500| Purchase H| Store H|
|          9|  25/9/2022|   700| Purchase I| Store I|
|         10| 30/10/2022|  1800| Purchase J| Store J|
|         11|  5/11/2022|  2200| Purchase K| Store K|
|         12| 10/12/2022|   900| Purchase L| Store L|
|         13|  15/1/2023|  4800| Purchase M| Store M|
|         14|  20/2/2023|   300| Purchase N| Store N|
|         15|  25/3/2023|  4200| Purchase O| Store O|
|         16|  30/4/2023|  2

In [10]:
df2.show()

+-----------+----------------+-----+-------+
|customer_id|transaction_date|value|  notes|
+-----------+----------------+-----+-------+
|          1|        1/1/2022| 1500| Note 1|
|          2|       15/2/2022| 2000| Note 2|
|          3|       20/3/2022| 1000| Note 3|
|          4|       10/4/2022| 2500| Note 4|
|          5|        5/5/2022| 1800| Note 5|
|          6|       10/6/2022| 1200| Note 6|
|          7|       15/7/2022|  700| Note 7|
|          8|       20/8/2022| 3000| Note 8|
|          9|       25/9/2022|  600| Note 9|
|         10|      30/10/2022| 1200|Note 10|
|         11|       5/11/2022| 1500|Note 11|
|         12|      10/12/2022|  800|Note 12|
|         13|       15/1/2023| 2000|Note 13|
|         14|       20/2/2023|  700|Note 14|
|         15|       25/3/2023| 1800|Note 15|
|         16|       30/4/2023| 1000|Note 16|
|         17|        5/5/2023|  400|Note 17|
|         18|       10/6/2023| 1500|Note 18|
|         19|       15/7/2023| 3000|Note 19|
|         

In [11]:
#print the schema of df1 and df2

df1.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- date_column: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- description: string (nullable = true)
 |-- location: string (nullable = true)



In [72]:
df2.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- transaction_date: string (nullable = true)
 |-- value: string (nullable = true)
 |-- notes: string (nullable = true)
 |-- quarter: date (nullable = true)




<details>
    <summary>Click here for Solution</summary>

```python

#print the schema of df1 and df2
    
df1.printSchema()
df2.printSchema()
```

</details>




#### Task 3: Add a new column to each dataframe

Add a new column named **year** to `df1` and **quarter** to `df2` representing the year and quarter of the data.

*Hint: use withColumn. Convert the date columns which are present as string to date before extracting the year and quarter information*




In [129]:
from pyspark.sql.functions import year, quarter, to_date,col,regexp_replace


df1 = df1.withColumn("date_column", regexp_replace("date_column",'/','-'))
#Add new column year to df1
df1 = df1.withColumn('year',year(to_date(df1['date_column'],'d-M-yyyy')))

df2 = df2.withColumn("transaction_date", regexp_replace("transaction_date",'/','-'))

#Add new column quarter to df2    
df2 = df2.withColumn('quarter',quarter(to_date(df2['transaction_date'],'d-M-yyyy')))



In [131]:
df1.show()

+-----------+-----------+------------------+----+
|customer_id|date_column|transaction_amount|year|
+-----------+-----------+------------------+----+
|          1|   1-1-2022|              5000|2022|
|          2|  15-2-2022|              1200|2022|
|          3|  20-3-2022|               800|2022|
|          4|  10-4-2022|              3000|2022|
|          5|   5-5-2022|              6000|2022|
|          6|  10-6-2022|              4500|2022|
|          7|  15-7-2022|               200|2022|
|          8|  20-8-2022|              3500|2022|
|          9|  25-9-2022|               700|2022|
|         10| 30-10-2022|              1800|2022|
|         11|  5-11-2022|              2200|2022|
|         12| 10-12-2022|               900|2022|
|         13|  15-1-2023|              4800|2023|
|         14|  20-2-2023|               300|2023|
|         15|  25-3-2023|              4200|2023|
|         16|  30-4-2023|              2600|2023|
|         17|   5-5-2023|               700|2023|


In [130]:
df2.show()

+-----------+----------------+-----------------+-------+
|customer_id|transaction_date|transaction_value|quarter|
+-----------+----------------+-----------------+-------+
|          1|        1-1-2022|             1500|      1|
|          2|       15-2-2022|             2000|      1|
|          3|       20-3-2022|             1000|      1|
|          4|       10-4-2022|             2500|      2|
|          5|        5-5-2022|             1800|      2|
|          6|       10-6-2022|             1200|      2|
|          7|       15-7-2022|              700|      3|
|          8|       20-8-2022|             3000|      3|
|          9|       25-9-2022|              600|      3|
|         10|      30-10-2022|             1200|      4|
|         11|       5-11-2022|             1500|      4|
|         12|      10-12-2022|              800|      4|
|         13|       15-1-2023|             2000|      1|
|         14|       20-2-2023|              700|      1|
|         15|       25-3-2023| 

In [112]:
df2.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- transaction_date: string (nullable = true)
 |-- value: string (nullable = true)
 |-- notes: string (nullable = true)
 |-- quarter: integer (nullable = true)




<details>
    <summary>Click here for Solution</summary>

```python
from pyspark.sql.functions import year, quarter

#Add new column year to df1
df1 = df1.withColumn('year', year(to_date('date_column','dd/MM/yyyy')))
    
#Add new column quarter to df2    
df2 = df2.withColumn('quarter', quarter(to_date('transaction_date','dd/MM/yyyy')))```

</details>





#### Task 4: Rename columns in both dataframes

Rename the column **amount** to **transaction_amount** in `df1` and **value** to **transaction_value** in `df2`.

*Hint: Use withColumnRenamed*


In [113]:
#Rename df1 column amount to transaction_amount
df1 = df1.withColumnRenamed('amount','transaction_amount')

#Rename df2 column value to transaction_value
df2 = df2.withColumnRenamed('value','transaction_value')


In [114]:
print(df1.printSchema(),'\n',df2)

root
 |-- customer_id: string (nullable = true)
 |-- date_column: string (nullable = true)
 |-- transaction_amount: string (nullable = true)
 |-- description: string (nullable = true)
 |-- location: string (nullable = true)
 |-- year: integer (nullable = true)

None 
 DataFrame[customer_id: string, transaction_date: string, transaction_value: string, notes: string, quarter: int]




<details>
    <summary>Click here for Solution</summary>

```python

#Rename df1 column amount to transaction_amount
df1 = df1.withColumnRenamed('amount', 'transaction_amount')
    
#Rename df2 column value to transaction_value
df2 = df2.withColumnRenamed('value', 'transaction_value')
```

</details>





#### Task 5: Drop unnecessary columns

Drop the columns **description** and **location** from `df1` and **notes** from `df2`.




In [115]:
#Drop columns description and location from df1
df1 = df1.drop('description','location')

#Drop column notes from df2
df2  = df2.drop('notes')





<details>
    <summary>Click here for Solution</summary>

```python

#Drop columns description and location from df1
df1 = df1.drop('description', 'location')
    
#Drop column notes from df2
df2 = df2.drop('notes')
```

</details>





#### Task 6: Join dataframes based on a common column

Join `df1` and `df2` based on the common column **customer_id** and create a new dataframe named `joined_df`.




In [116]:
df1.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- date_column: string (nullable = true)
 |-- transaction_amount: string (nullable = true)
 |-- year: integer (nullable = true)



In [45]:
df2.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- transaction_date: string (nullable = true)
 |-- transaction_value: string (nullable = true)
 |-- quarter: integer (nullable = true)



In [117]:
#join df1 and df2 based on common column customer_id

joined_df = df1.join(df2,on='customer_id',how='inner')

<details>
    <summary>Click here for Solution</summary>

```python

#join df1 and df2 based on common column customer_id
joined_df = df1.join(df2, 'customer_id', 'inner')
    
```

</details>


#### Task 7: Filter data based on a condition

Filter `joined_df` to include only transactions where "transaction_amount" is greater than 1000 and create a new dataframe named `filtered_df`.





In [118]:
# filter the dataframe for transaction amount > 1000
filtered_df = joined_df.filter(joined_df['transaction_amount']>1000)

In [119]:
filtered_df.show()

+-----------+-----------+------------------+----+----------------+-----------------+-------+
|customer_id|date_column|transaction_amount|year|transaction_date|transaction_value|quarter|
+-----------+-----------+------------------+----+----------------+-----------------+-------+
|          1|   1/1/2022|              5000|null|        1-1-2022|             1500|      1|
|          2|  15/2/2022|              1200|null|       15-2-2022|             2000|      1|
|          4|  10/4/2022|              3000|null|       10-4-2022|             2500|      2|
|          5|   5/5/2022|              6000|null|        5-5-2022|             1800|      2|
|          6|  10/6/2022|              4500|null|       10-6-2022|             1200|      2|
|          8|  20/8/2022|              3500|null|       20-8-2022|             3000|      3|
|         10| 30/10/2022|              1800|null|      30-10-2022|             1200|      4|
|         11|  5/11/2022|              2200|null|       5-11-2022|    

<details>
    <summary>Click here for Solution</summary>

```python

# filter the dataframe for transaction amount > 1000
filtered_df = joined_df.filter("transaction_amount > 1000")    
```

</details>


#### Task 8: Aggregate data by customer

Calculate the total transaction amount for each customer in `filtered_df` and display the result.

*Hint: Use sum from pyspark.sql.functions*


In [120]:

# group by customer_id and aggregate the sum of transaction amount
from pyspark.sql.functions import *
total_amount_per_customer = filtered_df.groupby('customer_id').agg({'transaction_amount':'sum'}).alias('total_amount')

#display the result
total_amount_per_customer.show()


+-----------+-----------------------+
|customer_id|sum(transaction_amount)|
+-----------+-----------------------+
|         51|                 4200.0|
|         15|                 4200.0|
|         54|                 1500.0|
|         11|                 2200.0|
|         69|                 5500.0|
|         42|                 1500.0|
|         73|                 1800.0|
|         87|                 4200.0|
|         64|                 2600.0|
|         30|                 1500.0|
|         34|                 1200.0|
|          8|                 3500.0|
|         22|                 1200.0|
|         28|                 2600.0|
|         85|                 1800.0|
|         16|                 2600.0|
|         52|                 2600.0|
|         99|                 4200.0|
|         96|                 2400.0|
|         43|                 3200.0|
+-----------+-----------------------+
only showing top 20 rows



In [121]:
total_amount_per_customer1  = filtered_df.groupBy('customer_id').agg(sum('transaction_amount').alias('total_amount'))
total_amount_per_customer1.show()

+-----------+------------+
|customer_id|total_amount|
+-----------+------------+
|         51|      4200.0|
|         15|      4200.0|
|         54|      1500.0|
|         11|      2200.0|
|         69|      5500.0|
|         42|      1500.0|
|         73|      1800.0|
|         87|      4200.0|
|         64|      2600.0|
|         30|      1500.0|
|         34|      1200.0|
|          8|      3500.0|
|         22|      1200.0|
|         28|      2600.0|
|         85|      1800.0|
|         16|      2600.0|
|         52|      2600.0|
|         99|      4200.0|
|         96|      2400.0|
|         43|      3200.0|
+-----------+------------+
only showing top 20 rows



<details>
    <summary>Click here for Solution</summary>

```python

from pyspark.sql.functions import sum
   
# group by customer_id and aggregate the sum of transaction amount

total_amount_per_customer = filtered_df.groupBy('customer_id').agg(sum('transaction_amount').alias('total_amount'))

#display the result
total_amount_per_customer.show()
```

</details>


#### Task 9: Write the result to a Hive table

Write `total_amount_per_customer` to a Hive table named **customer_totals**.


In [71]:
# Write total_amount_per_customer to a Hive table named customer_totals

total_amount_per_customer1.write.mode("overwrite").saveAsTable("customer_totals") #bunu total_amount_per_customer1 uzerinden yapamamin sebebi digerinde sumli gosterim enge


<details>
    <summary>Click here for Solution</summary>

```python

# Write total_amount_per_customer to a Hive table named customer_totals
total_amount_per_customer.write.mode("overwrite").saveAsTable("customer_totals")
```

</details>


#### Task 10: Write the filtered data to HDFS

Write `filtered_df` to HDFS in parquet format to a file named **filtered_data**.


In [ ]:
#Write filtered_df to HDFS in parquet format file filtered_data.parquet

filtered.write.mode("overwrite").parquet("filtered_data.parquet")


<details>
    <summary>Click here for Solution</summary>

```python

#Write filtered_df to HDFS in parquet format file filtered_data

filtered_df.write.mode("overwrite").parquet("filtered_data.parquet")
```

</details>


#### Task 11: Add a new column based on a condition

Add a new column named **high_value** to `df1` indicating whether the transaction_amount is greater than 5000. When the value is greater than 5000, the value of the column should be **Yes**. When the value is less than or equal to 5000, the value of the column should be **No**. 

*Hint: Use when and lit from pyspark.sql.functions


In [72]:
# Add new column with value indicating whether transaction amount is > 5000 or not

df1 = df1.withColumn('high_value',when(df1.transaction_amount >5000,lit("Yes")).otherwise(lit("No")))

In [73]:
df1.show(5)

+-----------+-----------+------------------+----+----------+
|customer_id|date_column|transaction_amount|year|high_value|
+-----------+-----------+------------------+----+----------+
|          1|   1/1/2022|              5000|null|        No|
|          2|  15/2/2022|              1200|null|        No|
|          3|  20/3/2022|               800|null|        No|
|          4|  10/4/2022|              3000|null|        No|
|          5|   5/5/2022|              6000|null|       Yes|
+-----------+-----------+------------------+----+----------+
only showing top 5 rows




<details>
    <summary>Click here for Solution</summary>

```python

from pyspark.sql.functions import when, lit

# Add new column with value indicating whether transaction amount is > 5000 or not
df1 = df1.withColumn("high_value", when(df1.transaction_amount > 5000, lit("Yes")).otherwise(lit("No")))
```

</details>


total_amount_per_customer#### Task 12: Calculate the average transaction value per quarter

Calculate and display the average transaction value for each quarter in `df2` and create a new dataframe named `average_value_per_quarter` with column `avg_trans_val`.

*Hint: Use avg from pyspark.sql.functions*


In [79]:
df2.groupby('quarter').agg(mean('transaction_value').alias('avg_trans_val')).show()

+-------+-------------+
|quarter|avg_trans_val|
+-------+-------------+
|   null|       1234.0|
+-------+-------------+



In [125]:
#calculate the average transaction value for each quarter in df2
average_value_per_quarter = df2.groupby('quarter').agg(avg('transaction_value').alias('avg_trans_val'))

#show the average transaction value for each quarter in df2    



In [126]:
average_value_per_quarter.show()

+-------+------------------+
|quarter|     avg_trans_val|
+-------+------------------+
|      1| 1111.111111111111|
|      3|1958.3333333333333|
|      4| 816.6666666666666|
|      2|            1072.0|
+-------+------------------+




<details>
    <summary>Click here for Solution</summary>

```python
from pyspark.sql.functions import avg

#calculate the average transaction value for each quarter in df2
average_value_per_quarter = df2.groupBy('quarter').agg(avg("transaction_value").alias("avg_trans_val"))

    
#show the average transaction value for each quarter in df2    
average_value_per_quarter.show()

```

</details>


#### Task 13: Write the result to a Hive table

Write `average_value_per_quarter` to a Hive table named **quarterly_averages**.


In [127]:
#Write average_value_per_quarter to a Hive table named quarterly_averages

average_value_per_quarter.write.mode("overwrite").saveAsTable("quarterly_averages")


<details>
    <summary>Click here for Solution</summary>

```python

#Write average_value_per_quarter to a Hive table named quarterly_averages

average_value_per_quarter.write.mode("overwrite").saveAsTable("quarterly_averages")

```

</details>


#### Task 14: Calculate the total transaction value per year

Calculate and display the total transaction value for each year in `df1` and create a new dataframe named `total_value_per_year` with column `total_transaction_val`.


In [133]:
# calculate the total transaction value for each year in df1.

total_value_per_year = df1.groupby('year').agg(sum('transaction_amount').alias('total_transaction_val'))
# show the total transaction value for each year in df1.
total_value_per_year.show()


+----+---------------------+
|year|total_transaction_val|
+----+---------------------+
|2025|              25700.0|
|2027|              25700.0|
|2023|              28100.0|
|2022|              29800.0|
|2026|              25700.0|
|2029|              25700.0|
|2030|               9500.0|
|2028|              25700.0|
|2024|              25700.0|
+----+---------------------+




<details>
    <summary>Click here for Solution</summary>

```python

# calculate the total transaction value for each year in df1.
total_value_per_year = df1.groupBy('year').agg(sum("transaction_amount").alias("total_transaction_val"))

# show the total transaction value for each year in df1.
total_value_per_year.show()

```

</details>


#### Task 15: Write the result to HDFS

Write `total_value_per_year` to HDFS in the CSV format to file named **total_value_per_year**.



In [ ]:
#Write total_value_per_year to HDFS in the CSV format
total_value_per_year.write.mode("overwrite)



<details>
    <summary>Click here for Solution</summary>

```python

#Write total_value_per_year to HDFS in the CSV format

total_value_per_year.write.mode("overwrite").csv("total_value_per_year.csv")

```

</details>


### Congratulations! You have completed the lab.
This practice project provides hands-on experience with data transformation and integration using PySpark. You've performed various tasks, including adding columns, renaming columns, dropping unnecessary columns, joining dataframes, and writing the results into both a Hive warehouse and an HDFS file system.


## Authors

Raghul Ramesh

Lavanya T S


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2023-09-01|0.1|Lavanya T S|Initial version|
|2023-09-08|0.2|Pornima More|QA pass with edits|


Copyright © 2023 IBM Corporation. All rights reserved.
